In [1]:
import boto3
import json
from concurrent.futures import ThreadPoolExecutor

### `Lambda`

In [ ]:
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

# TODO: open the zip file containing your Lambda function
with open('<lambda_func_zip>.zip', 'rb') as f:
    lambda_zip = f.read()

In [6]:
try:
    # If function hasn't yet been created, create it
    # TODO: replace the placeholders with your function specifics
    response = aws_lambda.create_function(
        FunctionName='<lambda_function_name>',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='<file_name>.<lambda_function_name>',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
        FunctionName='<lambda_function_name>',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']
lambda_arn

'arn:aws:lambda:us-east-1:591015244613:function:lambda_handler'

In [7]:
# can invoke serially:
r = aws_lambda.invoke(FunctionName='<lambda_function_name>',
                      InvocationType='RequestResponse',
                      Payload=json.dumps({'num_points':10000}))
json.loads(r['Payload'].read())

{'statusCode': 200, 'body': {'pi_estimate': 3.1384, 'num_points': 10000}}

In [ ]:
# ... or invoke the same function multiple times in parallel and gather the results:

# 1. write function to invoke our function for us and pass in data:
def invoke_function(data):
    r = aws_lambda.invoke(FunctionName='<lambda_function_name>',
                      InvocationType='RequestResponse',
                      Payload=json.dumps({'num_points':data}))
    return json.loads(r['Payload'].read())

# 2. Demo that lambda function will scale out if called concurrently on different threads locally
with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(invoke_function, [100000 for _ in range(10)])

output = [result for result in results]

In [31]:
# return avg of simulations
print(sum([i['body']['pi_estimate'] for i in output]) / len(output))

3.143235340481622
